In [ ]:
# Mengimpor pustaka PyTorch dan lainnya untuk manipulasi data dan visualisasi
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split  # Untuk membagi data ke dalam set latih dan uji
from sklearn.preprocessing import StandardScaler  # Untuk normalisasi data
from sklearn.metrics import accuracy_score  # Untuk menghitung akurasi
import matplotlib.pyplot as plt  # Untuk visualisasi hasil
from torch.utils.data import DataLoader, TensorDataset  # Untuk membuat DataLoader
from torch.autograd import Variable  # Untuk memungkinkan otomatisasi diferensiasi
import time  # Untuk mengukur waktu eksekusi
from fpdf import FPDF  # Untuk membuat laporan PDF


In [ ]:
# Membaca dataset winequality-white.csv dengan delimiter ;
df = pd.read_csv('winequality-white.csv', delimiter=';')

# Pisahkan fitur (X) dan target (y)
X = df.drop('quality', axis=1).values  # Semua kolom selain 'quality' sebagai fitur
y = df['quality'].values  # Kolom 'quality' sebagai target

# Normalisasi data fitur agar nilai berada dalam rentang yang sama
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Fit dan transform data fitur

# Membagi data menjadi set latih (80%) dan uji (20%)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Mengubah data menjadi tensor PyTorch agar bisa digunakan dalam pelatihan
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Membuat dataset Tensor untuk DataLoader
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

# Membuat DataLoader untuk batch training dan testing
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


In [ ]:
# Mendefinisikan kelas model MLP (Multilayer Perceptron) untuk klasifikasi
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_layers, hidden_neurons, activation_fn):
        super(MLP, self).__init__()
        layers = []  # List untuk menyimpan layer-layer

        prev_layer = input_dim  # Dimensi input adalah jumlah fitur

        # Menambahkan hidden layers sesuai jumlah yang ditentukan
        for _ in range(hidden_layers):
            layers.append(nn.Linear(prev_layer, hidden_neurons))  # Menambahkan layer Linear
            if activation_fn == 'relu':  # Jika fungsi aktivasi ReLU
                layers.append(nn.ReLU())  # Menambahkan ReLU
            elif activation_fn == 'sigmoid':  # Jika fungsi aktivasi Sigmoid
                layers.append(nn.Sigmoid())  # Menambahkan Sigmoid
            elif activation_fn == 'tanh':  # Jika fungsi aktivasi Tanh
                layers.append(nn.Tanh())  # Menambahkan Tanh
            elif activation_fn == 'softmax':  # Jika fungsi aktivasi Softmax
                layers.ap


In [ ]:
# Fungsi untuk melatih dan mengevaluasi model
def train_model(model, criterion, optimizer, epochs, train_loader, test_loader):
    train_losses = []  # Menyimpan loss selama pelatihan
    test_accuracies = []  # Menyimpan akurasi setiap epoch

    for epoch in range(epochs):
        model.train()  # Menyiapkan model untuk pelatihan
        running_loss = 0.0  # Variabel untuk menghitung loss selama satu epoch

        # Loop untuk setiap batch di train_loader
        for inputs, labels in train_loader:
            # Forward pass: menghitung output dari model
            outputs = model(inputs)
            loss = criterion(outputs, labels)  # Menghitung loss

            # Backward pass dan optimasi: memperbarui parameter model
            optimizer.zero_grad()  # Mengatur gradient ke 0 sebelum backward pass
            loss.backward()  # Menghitung gradien
            optimizer.step()  # Mengupdate parameter model

            running_loss += loss.item()  # Menambah loss per batch ke total loss
        
        # Menyimpan rata-rata loss untuk epoch ini
        train_losses.append(running_loss / len(train_loader))

        # Evaluasi model pada data uji
        model.eval()  # Menyiapkan model untuk evaluasi
        correct = 0  # Variabel untuk menghitung jumlah prediksi yang benar
        total = 0  # Variabel untuk menghitung total data

        with torch.no_grad():  # Menonaktifkan gradient calculation
            for inputs, labels in test_loader:
                outputs = model(inputs)  # Menghitung output model
             


In [ ]:
hidden_layers_options = [1, 2, 3]
hidden_neurons_options = [4, 8, 16, 32]
activation_functions = ['linear','relu', 'sigmoid', 'tanh', 'softmax']
epochs_options = [25, 50, 100]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [32, 64, 128]

best_accuracy = 0
best_params = {}

# Looping untuk mengevaluasi semua kombinasi hyperparameter
for hidden_layers in hidden_layers_options:
    for hidden_neurons in hidden_neurons_options:
        for activation_fn in activation_functions:
            for epochs in epochs_options:
                for lr in learning_rates:
                    for batch_size in batch_sizes:
                        print(f"Training with {hidden_layers} hidden layers, {hidden_neurons} neurons, {activation_fn} activation, {epochs} epochs, {lr} learning rate, {batch_size} batch size")
                        
                        # Membuat model dan optimizer
                        model = MLP(X_train.shape[1], hidden_layers, hidden_neurons, activation_fn)
                        criterion = nn.CrossEntropyLoss()
                        optimizer = optim.Adam(model.parameters(), lr=lr)
                        
                        # Train model
                        train_losses, test_accuracies = train_model(model, criterion, optimizer, epochs, train_loader, test_loader)
                        
                        # Evaluasi dan simpan hasil terbaik
                        final_accuracy = test_accuracies[-1]
                        if final_accuracy > best_accuracy:
                            best_accuracy = final_accuracy
                            best_params = {
                                'hidden_layers': hidden_layers,
                                'hidden_neurons': hidden_neurons,
                                'activation_fn': activation_fn,
                                'epochs': epochs,
                                'lr': lr,
                                'batch_size': batch_size
                            }
                        
print("Best Hyperparameters:", best_params)
print("Best Accuracy:", best_accuracy)


In [ ]:
# Fungsi untuk menyimpan hasil analisis dalam bentuk laporan PDF
def save_analysis_report(best_params, best_accuracy):
    pdf = FPDF()  # Memb
